In [62]:
include("../src/JuliaMBD.jl")

Main.JuliaMBD

In [63]:
using DifferentialEquations
using Plots
import .JuliaMBD

In [86]:
## MSD

function MSD(;M, D, k, g = 9.8)
    b = JuliaMBD.SubSystemBlock(:MSD)

    JuliaMBD.addparameter!(b, :M, M)
    JuliaMBD.addparameter!(b, :D, D)
    JuliaMBD.addparameter!(b, :k, k)
    JuliaMBD.addparameter!(b, :g, g)

    in1 = JuliaMBD.Inport(:in1)
    JuliaMBD.add!(b, in1)

    out1 = JuliaMBD.Outport(:out1)
    JuliaMBD.add!(b, out1)

    constant1 = JuliaMBD.Constant(value = :(M*g))
    JuliaMBD.add!(b, constant1)

    gain1 = JuliaMBD.Gain(K = :D)
    JuliaMBD.add!(b, gain1)

    gain2 = JuliaMBD.Gain(K = :k)
    JuliaMBD.add!(b, gain2)

    gain3 = JuliaMBD.Gain(K = :(1/M))
    JuliaMBD.add!(b, gain3)

    int1 = JuliaMBD.Integrator(sin = JuliaMBD.OutPort(:int1sin), sout = JuliaMBD.OutPort(:int1sout))
    JuliaMBD.add!(b, int1)

    int2 = JuliaMBD.Integrator(initialcondition = :(M*g/k), sin = JuliaMBD.OutPort(:int2sin), sout = JuliaMBD.OutPort(:int2sout))
    JuliaMBD.add!(b, int2)

    sum1 = JuliaMBD.Add(signs=[:+, :+, :-, :-])
    JuliaMBD.add!(b, sum1)

    JuliaMBD.LineSignal(in1.env[:out], sum1.env[:in1])
    JuliaMBD.LineSignal(constant1.env[:out], sum1.env[:in2])
    JuliaMBD.LineSignal(gain1.env[:out], sum1.env[:in3])
    JuliaMBD.LineSignal(gain2.env[:out], sum1.env[:in4])
    JuliaMBD.LineSignal(sum1.env[:out], gain3.env[:in])

    JuliaMBD.LineSignal(gain3.env[:out], int1.env[:in])
    JuliaMBD.LineSignal(int1.env[:out], int2.env[:in])
    JuliaMBD.LineSignal(int1.env[:out], gain1.env[:in])

    JuliaMBD.LineSignal(int2.env[:out], gain2.env[:in])
    JuliaMBD.LineSignal(int2.env[:out], out1.env[:in])

    JuliaMBD.addscope!(b, :x, out1.env[:in])
    b
end

MSD (generic function with 1 method)

In [87]:
## test

function Test(;M, D, k, f, p_cycle, p_width)
    b = JuliaMBD.SubSystemBlock(:Test)

    # JuliaMBD.addparameter!(b, :M, M)
    # JuliaMBD.addparameter!(b, :D, D)
    # JuliaMBD.addparameter!(b, :k, k)
    # JuliaMBD.addparameter!(b, :f, f)
    # JuliaMBD.addparameter!(b, :p_cycle, p_cycle)
    # JuliaMBD.addparameter!(b, :p_width, p_width)
    # JuliaMBD.addparameter!(b, :g, 9.8)

    msd = MSD(M=M, D=D, k=k)
    JuliaMBD.add!(b, msd)

    pulse = JuliaMBD.PulseGenerator(amplitude=f, period=p_cycle, pulsewidth=p_width, phasedelay=10.0)
    JuliaMBD.LineSignal(b.timeport, pulse.env[:time])
    JuliaMBD.add!(b, pulse)

    JuliaMBD.LineSignal(pulse.env[:out], msd.env[:in1])

    JuliaMBD.addscope!(b, :F, pulse.env[:out])
    b
end

Test (generic function with 1 method)

In [93]:
b = MSD(M=10, D=10, k=10)
JuliaMBD.expr_sfunc(b)

:(function MSD_sfunc(int1sin, int2sin, in1; M = 10, D = 10, k = 10, g = 9.8, time = 0)
      ()
      var"##989" = k
      var"##994" = 0.0
      var"##986" = D
      var"##980" = in1
      var"##983" = M * g
      var"##982" = var"##983"
      var"##992" = 1 / M
      begin
          var"##1012" = int2sin
          var"##1013" = int2sin
      end
      var"##987" = var"##1012"
      var"##988" = var"##989" * var"##987"
      begin
          var"##1007" = var"##988"
      end
      var"##981" = var"##1013"
      out1 = var"##981"
      begin
      end
      var"##998" = (nothing, nothing)
      begin
          var"##1005" = var"##982"
      end
      begin
          var"##1010" = int1sin
          var"##1011" = int1sin
      end
      var"##984" = var"##1011"
      var"##985" = var"##986" * var"##984"
      begin
          var"##1006" = var"##985"
      end
      var"##1002" = var"##1006"
      var"##996" = var"##1010"
      begin
          var"##1004" = var"##980"
      end
      var"

In [88]:
b = Test(M=10, D=10, k=10, f=10, p_cycle=20, p_width=50)

"Block(Test)"

In [89]:
eval(JuliaMBD.expr_sfunc(b))
eval(JuliaMBD.expr_ofunc(b))
eval(JuliaMBD.expr_ifunc(b))
m = JuliaMBD.ODEModel(
    b,
    eval(JuliaMBD.expr_odemodel_ifunc(b)),
    eval(JuliaMBD.expr_odemodel_sfunc(b)),
    eval(JuliaMBD.expr_odemodel_ofunc(b))
);

In [90]:
result = JuliaMBD.simulate(m, (0.0, 60.0))
plot(result)

UndefVarError: UndefVarError: M not defined